In [ ]:
!pip install nbimporter

In [1]:
import numpy as np
import pandas as pd
import nbimporter 
import matrix_factorization
np.random.seed(0)

Importing Jupyter notebook from matrix_factorization.ipynb


In [ ]:
# Downloading Movielens-100k
!curl -O http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip
!cd ml-100k/

In [ ]:
import zipfile
with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [2]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('./ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [4]:
ratings.shape

(943, 1682)

In [5]:
def get_sparsity(mat):
    print (str(n_users) + ' users')
    print (str(n_items) + ' items')
    sparsity = float(len(mat.nonzero()[0]))
    sparsity /= (mat.shape[0] * mat.shape[1])
    sparsity *= 100
    return sparsity

In [6]:
print ('Sparsity: {:4.2f}%'.format(get_sparsity(ratings)))

943 users
1682 items
Sparsity: 6.30%


In [7]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [8]:
train, test = train_test_split(ratings)

In [9]:
MF_SGD = matrix_factorization.ExplicitMF(train, 40, learning='sgd', verbose=True)
# iter_array = [1, 2, 5, 10, 25, 50, 100, 200]

iter_array = [10]
MF_SGD.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 10
	current iteration: 10
Train mse: 0.9179628673300955
Test mse: 1.0113053267188021


##################################################################################################################

In [10]:
train.shape

(943, 1682)

In [11]:
test.shape

(943, 1682)

In [12]:
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [13]:
get_sparsity(ratings)

943 users
1682 items


6.304669364224531

In [14]:
get_sparsity(train)

943 users
1682 items


5.710139043178159

In [15]:
get_sparsity(test)

943 users
1682 items


0.5945303210463734

# ####################################### GANS ###################################################

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data as t_data
import torchvision.datasets as datasets
from torchvision import transforms

In [17]:
batch_size = 5

In [ ]:
def make_some_noise(batch_size):
    return torch.rand(batch_size,100)

In [ ]:
make_some_noise(batch_size)

In [ ]:
# defining generator class

class generator(nn.Module):
    
    def __init__(self, inp, out):
        
        super(generator, self).__init__()
        
        self.net = nn.Sequential(
                                 nn.Linear(inp,300),
                                 nn.Tanh(),
                                 nn.Linear(300,1000),
                                 nn.Tanh(),
                                 nn.Linear(1000,800),
                                 nn.Tanh(),
                                 nn.Linear(800,out),
                                 nn.Sigmoid()
                                    )
        
    def forward(self, x):
        x = self.net(x)
        return x*5 # to get values in range [0,5]

In [ ]:
# defining discriminator class

class discriminator(nn.Module):
    
    def __init__(self, inp, out):
        
        super(discriminator, self).__init__()
        
        self.net = nn.Sequential(
                                 nn.Linear(inp,300),
                                 nn.Tanh(),
                                 nn.Linear(300,300),
                                 nn.Tanh(),
                                 nn.Linear(300,200),
                                 nn.Tanh(),
                                 nn.Linear(200,out),
                                 nn.Sigmoid()
                                    )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [ ]:
dis = discriminator(ratings.shape[1], 1)
gen = generator(100, ratings.shape[1])

In [ ]:
dis

In [ ]:
gen

In [ ]:
d_steps = 300
g_steps = 300

criteriond1 = nn.BCELoss()
optimizerd1 = optim.SGD(dis.parameters(), lr=0.0001, momentum=0.9)

criteriond2 = nn.BCELoss()
optimizerd2 = optim.SGD(gen.parameters(), lr=0.0001, momentum=0.9)

# printing_steps = 200

epochs = 50

In [ ]:
def get_random_batch(mat, batch_size=16):
    rand_rows = np.random.randint(mat.shape[0], size=batch_size)
#     print(mat.shape, rand_rows)
#     print(mat[rand_rows].shape)
    return mat[rand_rows]
    
get_random_batch(train)

In [ ]:
train = torch.autograd.Variable(torch.Tensor(train))

In [ ]:
import torch.nn.functional as F
eval_losses = []
for epoch in range(10):
#     print (epoch)

    # training discriminator
    gen.train()
    dis.train()
    for d_step in range(d_steps):
        dis.zero_grad()
        
        # training discriminator on real data
        real_rows = get_random_batch(train, batch_size)
        discriminator_real_outputs = dis(real_rows)
   
        dis_real_loss = criteriond1(discriminator_real_outputs, Variable(torch.ones(batch_size,1)))
    
        dis_real_loss.backward()

        # training discriminator on data produced by generator
        z_vector = make_some_noise(batch_size)
        #output from generator is generated        
        fake_rows = gen(z_vector).detach()
#         print(fake_rows[:20])
        dis_fake_out = dis(fake_rows)
        dis_fake_loss = criteriond1(dis_fake_out, Variable(torch.zeros(batch_size,1)))
        dis_fake_loss.backward()

        optimizerd1.step()
        
    # training generator
    for g_step in range(g_steps):
        gen.zero_grad()
        
        #generating data for input for generator
        z_vector = make_some_noise(batch_size)
        
        fake_rows = gen(z_vector)
#         print(fake_rows.shape, z_vector.shape)
#         print(fake_rows[:20])
        dis_out_gen_training = dis(fake_rows)
        gen_loss = criteriond2(dis_out_gen_training, Variable(torch.ones(batch_size,1)))
        gen_loss.backward()
        
        optimizerd2.step()

    # evaluation
    if epoch % 10: # todo- to change
        gen.eval()
        z_vector_eval = make_some_noise(128)
        fake_rows_eval = gen(z_vector_eval)
        real_rows_eval = get_random_batch(train, 128)
#         print(fake_rows[0][:10]) enable to see some results
        eval_loss = F.mse_loss(fake_rows_eval, real_rows_eval, reduction='sum')
        eval_losses.append(eval_loss)
        print('Epoch number {}. MSE distance between random real and fake samples {}'.format(epoch, eval_loss))
#         print('Epoch number {}. L1 distance between random real and fake samples {}'.format(epoch, torch.sum(torch.abs(fake_rows_eval - real_rows_eval))))
        

In [ ]:
import matplotlib.pyplot as plt

plt.plot(eval_losses)
plt.show()

In [ ]:
z_vector = make_some_noise(16)
fake_rows = gen(z_vector)

In [ ]:
# as we see generator produces very similar vectors 
fake_rows[:,:9]

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.init as init
from os.path import isfile, isdir, join
import os
# from tensorboard_logger import configure, log_value

In [ ]:
# lrD = 5e-4
# lrG = 5e-4
# batch_size = 100
# cuda = True
# epochs = 1000
# device = 5
# seed = 42
# nz = 100
# d_iter = 5
# g_iter = 1
# lamba = 1e-2 # constant for L2 penalty (diversity)
# name = "mnist-experiment"
# # configure("runs/run-" + args.name, flush_secs=5)
# torch.manual_seed(seed)
# # if cuda:
# # #     torch.cuda.set_device('cuda')
# #     torch.cuda.manual_seed(seed)
# # data_loader = torch.utils.data.DataLoader(
# # datasets.MNIST('../data', train=True, download=True,
# # transform=transforms.Compose([transforms.ToTensor(),])), batch_size=batch_size, shuffle=True)

In [22]:
lrD = 5e-4
lrG = 5e-4
batch_size = 100
cuda = True
epochs = 1000
device = 5
seed = 1
nz = 10
d_iter = 5
g_iter = 1
lamba = 1e-2 # constant for L2 penalty (diversity)
name = "mnist-experiment"
# configure("runs/run-" + args.name, flush_secs=5)
torch.manual_seed(seed)


data_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
    transforms.ToTensor(),
    ])), batch_size=batch_size, shuffle=True)

In [23]:
# length=6
# batch_size=5

In [31]:
class NetD(torch.nn.Module):
    def __init__(self, use_cuda=True):
        super(NetD, self).__init__()
        self.use_cuda = use_cuda
        # top
        self.t1 = torch.nn.Linear(28 * 28, 1024)
        # bottom
        self.b1 = torch.nn.Linear(28 * 28, 1024)
        # combined
        self.fc = torch.nn.Linear(2 * 1024, 28 * 28)
    def forward(self, xr, xf):
        # get filt
        filt = 1 - (xr * (xf >= 0.5).float()) - ((1-xr) * (xf < 0.5).float())
        # random swap
        idr = torch.multinomial(torch.Tensor([0.5,0.5]), xr.size(0), replacement=True)
        idrx = idr.float().unsqueeze(1).expand_as(xr)
        if self.use_cuda: 
            idrx = idrx.cuda()
        idrx = Variable(idrx)
        xt = xr * idrx + xf * (1 - idrx)
        xb = xr * (1 - idrx) + xf * idrx
        # top : real
        xt = F.relu(self.t1(xt))
        # bottom : fake
        xb = F.relu(self.b1(xb))
        # combined
        x = torch.cat((xt, xb), 1)
        x = F.tanh(self.fc(x))
        # apply filter, aggregate
        x = filt * x
        x = x.mean(dim = 1).squeeze()
        # use sign, because of swapping
        sgn = idr * 2 - 1
        if self.use_cuda: 
            sgn = sgn.cuda()
        sgn = Variable(sgn.float())
        x = sgn * x
        return x
        
netG = torch.nn.Sequential(
    torch.nn.Linear(nz, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 28 * 28),
    torch.nn.Sigmoid()
    )

# networks
netD = NetD(use_cuda=False)
print(netG)
print(netD)
optimizerG = optim.RMSprop(netG.parameters(), lr=lrG)
optimizerD = optim.RMSprop(netD.parameters(), lr=lrD)
one = torch.FloatTensor([1])
mone = one * -1

Sequential(
  (0): Linear(in_features=10, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=784, bias=True)
  (3): Sigmoid()
)
NetD(
  (t1): Linear(in_features=784, out_features=1024, bias=True)
  (b1): Linear(in_features=784, out_features=1024, bias=True)
  (fc): Linear(in_features=2048, out_features=784, bias=True)
)


In [26]:
# class NetD(torch.nn.Module):
#     def __init__(self, use_cuda=True):
#         super(NetD, self).__init__()
#         self.use_cuda = use_cuda
#         # top
#         self.t1 = torch.nn.Linear(length, 1024)
#         # bottom
#         self.b1 = torch.nn.Linear(length, 1024)
#         # combined
#         self.fc = torch.nn.Linear(2 * 1024, length)
#     def forward(self, xr, xf):
#         # get filt
# #         print("##########"*40)
#         filt = 1 - (xr * (xf >= 0.5).float()) - ((1-xr) * (xf < 0.5).float())
# #         if (filt == xr * xf).all():
# #             print('AAAAAAAAAAAAAAAAAAAAAAAAAA')
            
# #         print('xr & xf', xr * xf)
# #         print('filt', filt)
# #         print('xr.shape, xf.shape', xr.shape, xf.shape)
# #         print('filt.shape', filt.shape)
# #         print('xr', xr)
# #         print('xf', xf)
# #         print('filt', filt)
# # #         return filt
# #         # random swap
#         idr = torch.multinomial(torch.Tensor([0.5,0.5]), xr.size(0), replacement=True)
# #         print(idr.shape)
# #         print('idr', idr)
#         idrx = idr.float().unsqueeze(1).expand_as(xr)
# #         print('idrx', idrx)
# #         print('idrx.shape', idrx.shape)
# #         print('idrx', idrx[10:20, 100:200])
#         if self.use_cuda: 
#             idrx = idrx.cuda()
#         idrx = Variable(idrx)
# #         print('xr.shape', xr.shape)
# #         print('xr', xr[10:20, 100:200])
# #         print('xr*idrx.shape', (xr*idrx).shape)
# #         print('xr*idrx', (xr*idrx)[10:20, 100:200])
# #         print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', xr * idrx == xr)
# #         for c in xr * idrx == xr:
# #             print(c)
#         xt = xr * idrx + xf * (1 - idrx)
#         xb = xr * (1 - idrx) + xf * idrx
# #         print('xt', xt)
# #         print('xb', xb)
#         # top : real
#         xt = F.relu(self.t1(xt))
#         # bottom : fake
#         xb = F.relu(self.b1(xb))
#         # combined
# #         print(xt.shape, xb.shape)
#         x = torch.cat((xt, xb), 1)
# #         print('x', x)
#         x = torch.tanh(self.fc(x))
# #         print('xxxx', x.shape)
# #         print('x[:, :10]', x[:10, :10])
# #         print('filt', filt[:10, :10])
#         # apply filter, aggregate
# #         print('x[:, :10]', x[:10, :10])
# #         print('filt', filt[:10, :10])
#         x = filt * x
# #         print('x', x)
# #         print('x[:, :10]', x[:10, :10])
# #         print('xxxx', x.shape)
# #         print('x', x[:10, :10])
# #         print(x.mean(dim = 1).shape, x.mean(dim = 1))
#         x = x.mean(dim = 1).squeeze()
# #         print('x', x)
# #         print('xxxx', x.shape)
#         # use sign, because of swapping
#         sgn = idr * 2 - 1
#         if self.use_cuda: 
#             sgn = sgn.cuda()
#         sgn = Variable(sgn.float())
#         x = sgn * x
# #         print('x', x)
# #         print("##########"*40)
#         return x

In [ ]:
# # latent_vec_size = 100
# # vec_size = 1000

# netG = torch.nn.Sequential(
#     torch.nn.Linear(nz, 1024),
#     torch.nn.ReLU(),
#     torch.nn.Linear(1024, length),
#     torch.nn.Sigmoid()
#     )

In [ ]:
# # networks
# netD = NetD(False)
# print(netG)
# print(netD)
# optimizerG = optim.RMSprop(netG.parameters(), lr=lrG)
# optimizerD = optim.RMSprop(netD.parameters(), lr=lrD)
# one = torch.FloatTensor([1])
# mone = one * -1

In [32]:
one = torch.FloatTensor([1])
mone = one * -1
cuda = False
if cuda is True:
    netD.cuda()
    netG.cuda()
    one, mone = one.cuda(), mone.cuda()

In [33]:
for p in netD.parameters(): # reset requires_grad
    p.requires_grad = True #
    
for p in netG.parameters(): # reset requires_grad
    p.requires_grad = True #

In [34]:
# batch_size = 5

In [35]:
def getRealSample(length=6):
     return Variable(torch.IntTensor(np.random.choice([0, 1], size=(batch_size, length))))

In [36]:
getRealSample()

tensor([[0, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 1, 1],
        [0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 1],
        [1, 0, 0, 1, 1, 1],
        [1, 1, 1, 1, 0, 0],
        [0, 1, 0, 1, 0, 1],
        [0, 1, 0, 1, 1, 1],
        [0, 1, 0, 1, 1, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 1],
        [1, 1, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 0],
        [1, 0, 1, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 0, 0],
        [1, 1, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 1, 1, 1, 1],
        [0, 1, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0],
        [1, 0, 1, 1, 0, 1],
        [0, 1, 1, 1, 0, 0],
        [1, 1, 0, 1,

In [37]:
cuda

False

In [ ]:

gen_iterations = 0
for epoch in range(epochs):
    data_iter = iter(data_loader)
    i = 0
    while i < len(data_loader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update
        d_iter = d_iter
        j = 0
        while j < d_iter and i < len(data_loader):
            j += 1
            # load real data
            i += 1
            X, _ = data_iter.next()
            X = X.view(X.size(0), -1)
            X = (X >= 0.5).float()
            if cuda: 
                X = X.cuda()
            real = Variable(X)
            # generate fake data
            noise = torch.randn(batch_size, nz)
            if cuda: 
                noise = noise.cuda()
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            # compute gradient, take step
            netD.zero_grad()
            out = netD(real, fake)
            outputD = torch.mean(out) + lamba * out.norm()
            stdD = torch.std(out)
            outputD.backward(mone)
            optimizerD.step()
        ############################
        # (2) Update G network
        ###########################
        g_iter = g_iter
        j = 0
        while j < g_iter and i < len(data_loader):
            j += 1
            for p in netD.parameters():
                p.requires_grad = False # to avoid computation
            netG.zero_grad()
            
            # load real data
            i += 1
            X, _ = data_iter.next()
            X = X.view(X.size(0), -1)
            X = (X >= 0.5).float()
            if cuda: 
                X = X.cuda()
            real = Variable(X)
            
        # update generator
        noise = torch.randn(batch_size, nz)
        if cuda: 
            noise = noise.cuda()
        noisev = Variable(noise)
        fake = netG(noisev)
        out = netD(real, fake)
        outputG = torch.mean(out) + lamba * out.norm()
        stdG = torch.std(out)
        outputG.backward(one)
        optimizerG.step()
        
        gen_iterations += 1
        
        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f '% (epoch, epochs, i, len(data_loader), gen_iterations, outputD.item(), outputG.item()))
        print('output_D', outputD.item(), gen_iterations)
        print('output_G', outputG.item(), gen_iterations)
        print('std_D', stdD.item(), gen_iterations)
        print('std_G', stdG.item(), gen_iterations)
        
        if gen_iterations % 100 == 0:
            if not isdir('./images/{0}'.format(name)):
                os.makedirs('./images/{0}'.format(name))
            real = real.data[0:100,:]
            real = real.view(real.size(0), 1, 28, 28)
            vutils.save_image(real, './images/{0}/real_samples.png'.format(name, gen_iterations))
            noise = torch.randn(min(100, batch_size), nz)
            if cuda: 
                noise = noise.cuda()
            fake = netG(Variable(noise, volatile=True))
            # fake = (fake.data >= 0.5).float()
            R = torch.rand(fake.size())
            fake = (fake.data.cpu() >= R).float()
            fake = fake.view(fake.size(0), 1, 28, 28)
            vutils.save_image(fake, './images/{0}/fake_samples_{1}.png'.format(name, gen_iterations))
    # do checkpointing
    if not isdir('./checkpoint/{0}'.format(name)):
        os.makedirs('./checkpoint/{0}'.format(name))
    torch.save(netG.state_dict(), './checkpoint/{0}/netG_epoch_{1}.pth'.format(name, epoch))
    torch.save(netD.state_dict(), './checkpoint/{0}/netD_epoch_{1}.pth'.format(name, epoch))

C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and no

[0/1000][6/600][1] Loss_D: 0.153966 Loss_G: 0.154785 
output_D 0.15396621823310852 1
output_G 0.1547846645116806 1
std_D 0.056182604283094406 1
std_G 0.06456294655799866 1


C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and no

[0/1000][12/600][2] Loss_D: 0.142314 Loss_G: 0.153394 
output_D 0.1423138827085495 2
output_G 0.15339423716068268 2
std_D 0.0769490972161293 2
std_G 0.06827173382043839 2


C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and no

...

C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and no

[2/1000][156/600][226] Loss_D: 0.175269 Loss_G: 0.175038 
output_D 0.1752692610025406 226
output_G 0.1750376969575882 226
std_D 0.04086572676897049 226
std_G 0.03762180358171463 226


C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and no

[2/1000][162/600][227] Loss_D: 0.172492 Loss_G: 0.177579 
output_D 0.17249180376529694 227
output_G 0.17757904529571533 227
std_D 0.03659065067768097 227
std_G 0.040397677570581436 227


C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\davidt\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [43]:
os.makedirs('./checkpoint/{0}'.format(name))

In [ ]:
# gen_iterations = 0
# for epoch in range(epochs):
# #     data_iter = iter(data_loader)
#     i = 0
#     while i < 10:
#         ############################
#         # (1) Update D network
#         ###########################
#         for p in netD.parameters(): # reset requires_grad
#             p.requires_grad = True # they are set to False below in netG update
#         d_iter = d_iter
#         j = 0
#         while j < d_iter and i < 10:
#             j += 1
#             # load real data
#             i += 1
#             X = getRealSample()
#             X = X.view(X.size(0), -1).float()
# #             X = (X >= 0.5).float()
#             if cuda: 
#                 X = X.cuda()
#             real = Variable(X, requires_grad=True)
#             # generate fake data
#             noise = torch.randn(batch_size, nz)
#             if cuda: 
#                 noise = noise.cuda()
#             with torch.no_grad():
#                 noisev = Variable(noise) # totally freeze netG
#             fake = (Variable(netG(noisev).data, requires_grad=True) > 0.5).float()
#             # compute gradient, take step
#             netD.zero_grad()
#             print(real, fake)
#             out = netD(real, fake)
# #             print(real.shape, fake.shape, out.shape)
#             outputD = torch.mean(out) + lamba * out.norm()
#             stdD = torch.std(out)
# #             print('outputD.shape', outputD)
#             outputD.backward(mone)
#             optimizerD.step()
# #             break
# #         break
# #     break

#             g_iter = g_iter
#             j = 0
#             while j < g_iter and i < 10:
#                 j += 1
#                 for p in netD.parameters():
#                     p.requires_grad = False # to avoid computation
#                 netG.zero_grad()
#                 # load real data
#                 i += 1
#                 X = getRealSample()
#                 X = X.view(X.size(0), -1)
# #                 X = (X >= 0.5).float()
#                 if cuda: 
#                     X = X.cuda()
#                 real = Variable(X.float(), requires_grad=True)
#                 # update generator
#                 noise = torch.randn(batch_size, nz)
#                 if cuda: 
#                     noise = noise.cuda()
#                 noisev = Variable(noise)
#                 fake = Variable((netG(noisev) >=0.5).float(), requires_grad=True)
#                 print("####"*40)
#                 print('REEEEAL', real)
#                 print('FAAAKE', fake)
#                 print("####"*40)
# #                 print(real.shape, fake.shape)
#                 out = netD(real, fake)
#                 outputG = torch.mean(out) + lamba * out.norm()
#                 stdG = torch.std(out)
#                 outputG.backward(one)
#                 optimizerG.step()
#                 gen_iterations += 1

#             print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f ' % (epoch, epochs, i, 10, gen_iterations, outputD.data.item(), outputG.data.item()))
#             print('output_D', outputD.data.item(), gen_iterations)
#             print('output_G', outputG.data.item(), gen_iterations)
#             print('std_D', stdD.data.item(), gen_iterations)
#             print('std_G', stdG.data.item(), gen_iterations)
# #             if gen_iterations % 100 == 0:
# #                 if not isdir('./images/{0}'.format(name)):
# #                     os.mkdir('./images/{0}'.format(name))
# #                 real = real.data[0:100,:]
# #                 real = real.view(real.size(0), 1, 28, 28)
# #                 vutils.save_image(real, './images/{0}/real_samples.png'.format(name, gen_iterations))
# #                 noise = torch.randn(min(100, batch_size), nz)
# #                 if cuda: 
# #                     noise = noise.cuda()
# #                 fake = netG(Variable(noise, volatile=True))
# #                 # fake = (fake.data >= 0.5).float()
# #                 R = torch.rand(fake.size())
# #                 fake = (fake.data.cpu() >= R).float()
# #                 fake = fake.view(fake.size(0), 1, 28, 28)
# #                 vutils.save_image(fake, './images/{0}/fake_samples_{1}.png'.format(name, gen_iterations))

# #             # do checkpointing
# #             if not isdir('./checkpoint/{0}'.format(name)):
# #                 os.mkdir('./checkpoint/{0}'.format(name))
# #             torch.save(netG.state_dict(), './checkpoint/{0}/netG_epoch_{1}.pth'.format(name, epoch))
# #             torch.save(netD.state_dict(), './checkpoint/{0}/netD_epoch_{1}.pth'.format(name, epoch))


In [ ]:
batch_size